In [3]:
import pymongo
import os
import pandas as pd


In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covidcrypto_db

In [9]:
bitcoin_file = os.path.join("Resources", "Bitcoin", "rev_bitcoin2020data.csv")
bitcoin_df = pd.read_csv(bitcoin_file)
bitcoin_df.head()

,Unnamed: 0,Timestamp,Block,Volume_(BTC),Volume_(Currency),Weighted_Price
0,0,2020-01-01,4202896,0.669749,"$4,807.37","$7,177.87"
1,1,2020-01-02,4204336,0.510742,"$3,637.32","$7,121.64"
2,2,2020-01-03,4205776,12.028600,"$88,234.41","$7,335.39"
3,3,2020-01-04,4207216,5.121394,"$37,530.58","$7,328.20"
4,4,2020-01-05,4208656,0.020840,$154.70,"$7,423.24"


In [15]:
daywise_file = os.path.join("Resources", "COVID-19","day_wise.csv")
daywise_df = pd.read_csv(daywise_file)
daywise_df = daywise_df.rename(columns={"Date": "Timestamp"})




In [17]:
merge_df = daywise_df.merge(bitcoin_df, how='outer', on='Timestamp')

,Timestamp,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries,Unnamed: 0,Block,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2020-01-22,555.0,17.0,28.0,510.0,0.0,0.0,0.0,3.06,5.05,60.71,6.0,21,4233136,1.600519,"$13,816.20","$8,632.32"
1,2020-01-23,654.0,18.0,30.0,606.0,99.0,1.0,2.0,2.75,4.59,60.00,8.0,22,4234576,35.503468,"$297,465.93","$8,378.50"
2,2020-01-24,941.0,26.0,36.0,879.0,287.0,8.0,6.0,2.76,3.83,72.22,9.0,23,4236016,516.031518,"$4,334,633.06","$8,399.94"
3,2020-01-25,1434.0,42.0,39.0,1353.0,493.0,16.0,3.0,2.93,2.72,107.69,11.0,24,4237456,0.051810,$429.69,"$8,293.57"
4,2020-01-26,2118.0,56.0,52.0,2010.0,684.0,14.0,13.0,2.64,2.46,107.69,13.0,25,4238896,7.383556,"$62,407.28","$8,452.20"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,2020-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252,4565776,17.606331,"$180,137.29","$10,231.39"
253,2020-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253,4567216,7.882901,"$81,215.17","$10,302.70"
254,2020-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254,4568656,0.648506,"$6,658.14","$10,266.89"
255,2020-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,4570096,2.391694,"$24,728.99","$10,339.53"


In [20]:
for index, row in merge_df.iterrows():
    db.data.insert_one(
        {
            "date": row["Timestamp"],
            "bitcoin_price": row["Weighted_Price"],
            "world_new_cases": row["New cases"],
            "world_new_deaths": row["New deaths"],
            "world_new_recovered": row["New recovered"],
            "world_confirmed": row["Confirmed"],
            "world_deaths": row["Deaths"],
            "world_recovered": row["Recovered"],
            "world_active": row["Active"],       
            "countries": []
        }
    )

In [21]:
csv_file = os.path.join("Resources", "COVID-19", "full_grouped.csv")
full_df = pd.read_csv(csv_file)
full_df.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa


In [ ]:
for index, row in df.iterrows():
    db.data.update_one({'date': str(row["Date"])},
                       {'$push': {"countries": {
                           "country_name": row["Country/Region"],
                           "country_confirmed": row["Confirmed"],
                           "country_deaths": row["Deaths"],
                           "country_recovered": row["Recovered"],
                           "country_active": row["Active"],
                           "country_new_cases": row["New cases"],
                           "country_new_deaths": row["New deaths"],
                           "country_new_recovered": row["New recovered"],
                               }
                           }
                       })